In [383]:
import copy, configparser, datetime, glob, re
from dateutil import parser
import ACILib

#Reading current config from file
#Docker WORKDIR is /HTTSDashboard/Apps/
config = configparser.ConfigParser()
config.read('./tacconfig.ini')
shift = 'apjc'
tech_strip = 'dcrs'

InterestWorkgroup = [wg.upper() for wg in config[shift]['{}_{}'.format('workgroup',tech_strip)].split(',') if wg]

start_date = '2020-02-27'
end_date = datetime.datetime.today().strftime("%Y-%m-%d")

##### Path in Container setup ######
flask_container_path = ''
notebook_container_path = '/home/jovyan'
current_container_path = notebook_container_path
logfilename = current_container_path + '/HTTSDashboard/logs/RabbitMQ_DCRS_Event.log*'
#logfilename = current_container_path + '/HTTSDashboard/logs/RabbitMQ_SV_Event.log*'
logfilelist = glob.glob(logfilename)

event_path = '/HTTSDashboard/logs/TAC/'+shift.upper()+'/DCRS/events/'
#event_path = '/HTTSDashboard/logs/TAC/'+shift.upper()+'/SV/events/'

reportpath = '/HTTSDashboard/logs/TAC/'+shift.upper()+'/DCRS/report/'
#reportpath = '/HTTSDashboard/logs/TAC/'+shift.upper()+'/ACI/report/'

acceptfilename = current_container_path + event_path + '*_AcceptEvent.txt'
acceptfilelist = glob.glob(acceptfilename)
inqueuefilename = current_container_path + event_path + '*_InQueueEvent.txt'
inqueuefilelist = glob.glob(inqueuefilename)

print("InterestWorkgroup:{}".format(InterestWorkgroup))
AcceptEventsFromFile = ACILib.FindAcceptEventsFromEventFile(acceptfilelist)
UpdateTime, InQueueEventsFromFile = ACILib.FindInQueueEventsFromEventFile(inqueuefilelist)

UpdateTime, InQueueEventsFromFile[2020][-10:], AcceptEventsFromFile[2020][-10:]

InterestWorkgroup:['APT-DCSW']


('2020-06-26 07:03:04',
 ['689371653-~3-~WW-DCRS-~2020-06-26 04:48:54',
  '689371168-~1-~UC-~2020-06-26 04:51:07',
  '689336958-~3-~WW-DCRS-~2020-06-26 04:52:01',
  '689371688-~2-~WW-DCRS-~2020-06-26 04:59:58',
  '689371391-~3-~WW-DCRS-~2020-06-26 05:01:03',
  '689371391-~3-~WW-DCRS-~2020-06-26 05:31:11',
  '689371801-~3-~WW-DCRS-~2020-06-26 05:36:48',
  '689371809-~3-~WW-DCRS-~2020-06-26 05:38:39',
  '689371831-~3-~WW-DCRS-~2020-06-26 05:43:49',
  '689371835-~3-~WW-DCRS-~2020-06-26 05:45:26'],
 ['689371780-~3-~mahirani-~HTTS-INDIA-DCSW-~2020-06-26 05:35:25',
  '689371817-~3-~lowtouch-~VIRTUAL-~2020-06-26 05:40:08',
  '689366236-~3-~chaslu-~APT-DCSW-~2020-06-26 05:42:09',
  '689371513-~2-~joeunder-~UC_WORKGROUP-~2020-06-26 05:45:37',
  '689371801-~3-~blockett-~APT-DCSW-~2020-06-26 05:48:23',
  '689371391-~3-~navchaud-~APT-DCSW-~2020-06-26 05:49:58',
  '689241191-~3-~jleschni-~APT-DCSW-~2020-06-26 05:51:38',
  '689371782-~3-~koiwata-~JAPAN-NEXUS-~2020-06-26 05:52:10',
  '689371831-~3-~j

In [386]:
InQueueEventsFromFile[2020][-10:]

def ConvertInQueueEventsByDate(start_date='2020-02-27',end_date=datetime.datetime.today().strftime("%Y-%m-%d"),InQueueEventsFromFile={}):    
    '''
    Input : 2020_InQueueEvent.txt
    No Shift check needed as the InQueue event are from APJC shift period
    {'All': ['689154452-~1-~FTS-~2020-06-09 00:02:18',
      '689248026-~3-~CX-APJC-BLR-ACI-SSPT-~2020-06-09 00:14:11',
      '689247673-~3-~FTS-~2020-06-09 00:28:44',
      '688563612-~1-~FTS-~2020-06-09 00:59:46',
      '689248319-~3-~CX-APJC-SYD-ACI-SSPT-~2020-06-09 01:45:19',
      '689211356-~3-~UC-~2020-06-09 02:16:22',
      '689248637-~3-~CX-APJC-BLR-ACI-SSPT-~2020-06-09 03:05:42',
      '689206025-~2-~CX-APJC-BLR-ACI-SSPT-~2020-06-09 03:18:10',
      '689241901-~3-~CX-APJC-BLR-ACI-SSPT-~2020-06-09 03:30:13',
      '689237505-~3-~UC-~2020-06-09 03:44:42',
      '689190081-~3-~CX-APJC-BLR-ACI-SSPT-~2020-06-09 03:50:47',
      '689248857-~3-~CX-APJC-SYD-ACI-SSPT-~2020-06-09 04:05:49',
      '689076996-~3-~CX-APJC-BLR-ACI-SSPT-~2020-06-09 04:08:11',
      '689248966-~3-~CX-APJC-BLR-ACI-SSPT-~2020-06-09 04:46:13',
      '689249154-~3-~CX-APJC-SYD-ACI-SSPT-~2020-06-09 05:38:24'],
     'FTS': ['689154452', '689247673', '688563612'],
     'CX-APJC-BLR-ACI-SSPT': ['689248026',
      '689248637',
      '689206025',
      '689241901',
      '689190081',
      '689076996',
      '689248966'],
     'CX-APJC-SYD-ACI-SSPT': ['689248319', '689248857', '689249154'],
     'UC': ['689211356', '689237505']}
     '''
    InQueueEventsByDate = {} # {'2020-05-22,{'CX-APJC-BLR-ACI-SSPT':[689148383],'CX-APJC-SYD-ACI-SSPT':[689148487]}}
    
    for year,events in InQueueEventsFromFile.items():
        if year < parser.parse(start_date).year:
            continue
        elif year > parser.parse(end_date).year:
            break
        for event in events:
            caseno,casesev,queue,timestamp = event.split('-~')
            #print(" {} - {}".format(timestamp,end_date))
            if parser.parse(timestamp) < parser.parse(start_date):
                continue
            elif parser.parse(timestamp) >= parser.parse(end_date)+datetime.timedelta(days=1):
                break
            date = parser.parse(timestamp).strftime("%Y-%m-%d")
            if date not in InQueueEventsByDate.keys():
                InQueueEventsByDate[date] = {}
                InQueueEventsByDate[date]['All'] = []
                InQueueEventsByDate[date]['AllCases'] = []
                
            if queue not in InQueueEventsByDate[date]:
                InQueueEventsByDate[date][queue] = []
            InQueueEventsByDate[date][queue].append(caseno)
            InQueueEventsByDate[date]['All'].append(event)
            InQueueEventsByDate[date]['AllCases'].append(caseno)
            
    return InQueueEventsByDate

InQueueEventsByDate = ConvertInQueueEventsByDate(start_date=start_date,InQueueEventsFromFile=InQueueEventsFromFile)
InQueueEventsByDate['2020-06-26']

{'All': ['689363887-~2-~FTS-~2020-06-26 00:00:12',
  '689361345-~1-~FTS-~2020-06-26 00:00:33',
  '689368439-~3-~WW-DCRS-~2020-06-26 00:00:48',
  '689351990-~2-~FTS-~2020-06-26 00:01:32',
  '689370502-~2-~FTS-~2020-06-26 00:04:56',
  '689361464-~1-~FTS-~2020-06-26 00:07:39',
  '689370638-~4-~WW-DCRS-~2020-06-26 00:17:22',
  '689361707-~3-~WW-DCRS-~2020-06-26 00:18:50',
  '689367297-~1-~WW-DCRS-~2020-06-26 00:25:02',
  '689313896-~3-~WW-DCRS-~2020-06-26 00:28:43',
  '689370256-~3-~WW-DCRS-~2020-06-26 00:30:07',
  '689370671-~3-~WW-DCRS-~2020-06-26 00:30:46',
  '689370676-~2-~WW-DCRS-~2020-06-26 00:31:29',
  '689313896-~3-~WW-DCRS-~2020-06-26 00:31:59',
  '689370676-~2-~WW-DCRS-~2020-06-26 00:37:41',
  '689370714-~2-~WW-DCRS-~2020-06-26 00:42:44',
  '689361657-~1-~UC-~2020-06-26 00:42:55',
  '689249756-~2-~WW-DCRS-~2020-06-26 00:46:06',
  '689370676-~2-~WW-DCRS-~2020-06-26 00:46:28',
  '689331072-~3-~WW-DCRS-~2020-06-26 00:47:31',
  '689370732-~3-~WW-DCRS-~2020-06-26 00:49:50',
  '6893707

In [388]:
AcceptEventsByDate = {} # {'2020-05-22',{caseno:[[zdazhi,tetration],[zmeng,HTTS]]}}
AcceptCaseSevByDate = {} # {'2020-05-22',{'1':[6xxxxx,6xxxxxx,'2':[6xxxxx,6xxxxxx]]}

def ConvertAcceptEventsByDate(start_date='2020-02-27',end_date=datetime.datetime.today().strftime("%Y-%m-%d"),AcceptEventsFromFile=[]):
    '''
    Input: 2020_AcceptEvent.txt
    No Shift check is needed as the AcceptEvent are from APJC shift
    Output:
    {'689241333': [['yiding2', 'JAPAN-ACI']],
         '689246419': [['junwa', 'UC_WORKGROUP']],
         '689247993': [['raghb', 'APT-ACI-SOLUTIONS2']],
         '689248026': [['reperuma', 'APT-ACI-SOLUTIONS2']],
         '689242958': [['tskanai', 'JAPAN-ACI']],
         '688563612': [['tonzeng', 'APT-ACI-SOLUTIONS']],
         '689248175': [['juoishi', 'JAPAN-HTTS-FLETS']],
         '689248248': [['junwa', 'APT-ACI-SOLUTIONS']],
         '689248290': [['koiwata', 'JAPAN-NEXUS']],
         '689248319': [['siddhp', 'APT-ACI-SOLUTIONS']],
         '689211356': [['shubhada', 'UC_WORKGROUP']],
         '689230467': [['yuxuliu', 'UC_WORKGROUP']],
         '689248577': [['yuxuliu', 'APT-GC-Solution']],
         '689248444': [['ykamimur', 'JAPAN-ACI']],
         '689247673': [['gauvasud', 'APT-ACI-SOLUTIONS2']],
         '689248613': [['jawalia', 'APT-ACI-SOLUTIONS2'],
          ['huaizhao', 'APT-GC-Solution']],
         '689206025': [['jawalia', 'APT-ACI-SOLUTIONS2']],
         '689248692': [['fushuang', 'APT-GC-Solution']],
         '689241901': [['maveer', 'APT-ACI-SOLUTIONS2']],
         '689190081': [['deepaky', 'APT-ACI-SOLUTIONS2']],
         '689248680': [['ttaoda', 'JAPAN-HTTS-FLETS']],
         '689220318': [['dawang', 'APT-GC-Solution']],
         '689076996': [['hethakur', 'APT-ACI-SOLUTIONS2']],
         '689248849': [['pli3', 'APT-GC-Solution']],
         '689248974': [['mkamijo', 'JAPAN-ACI']],
         '689248966': [['amikum', 'GCE-ACI-Solutions']],
         '689249029': [['yushimaz', 'JAPAN-ACI']],
         '689171782': [['krajasel', 'UC_WORKGROUP']],
         '689249154': [['tonzeng', 'APT-ACI-SOLUTIONS']],
         '689154452': [['amikum', 'GCE-ACI-Solutions']]}
    '''
    AcceptEventsByDate = {}
    AcceptCaseSevByDate = {}
    
    for year,events in AcceptEventsFromFile.items():
        if year < parser.parse(start_date).year:
            continue
        elif year > parser.parse(end_date).year:
            break
        for event in events:
            caseno,casesev,ccoid,workgroup,timestamp = event.split('-~')
            if parser.parse(timestamp) < parser.parse(start_date):
                continue
            elif parser.parse(timestamp) > parser.parse(end_date)+datetime.timedelta(days=1):
                break
            date = parser.parse(timestamp).strftime("%Y-%m-%d")
            if date not in AcceptEventsByDate.keys():
                AcceptEventsByDate[date] = {}
                AcceptCaseSevByDate[date] = {}
                AcceptCaseSevByDate[date]['1'] = []
                AcceptCaseSevByDate[date]['2'] = []
                AcceptCaseSevByDate[date]['3'] = []
                AcceptCaseSevByDate[date]['4'] = []
                
            if caseno not in AcceptEventsByDate[date].keys():
                AcceptEventsByDate[date][caseno] = []
            if ccoid != 'lowtouch':
                AcceptEventsByDate[date][caseno].append([ccoid,workgroup])
                AcceptCaseSevByDate[date][casesev].append(caseno)
                
    return AcceptEventsByDate,AcceptCaseSevByDate
    
RawAcceptEventsByDate,AcceptCaseSevByDate = ConvertAcceptEventsByDate(AcceptEventsFromFile=AcceptEventsFromFile) 
RawAcceptEventsByDate['2020-06-26'],AcceptCaseSevByDate['2020-06-26']

({'689370542': [['sthomasg', 'CO-HARDWARE'], ['annagar', 'APAC-GGN-HARDWARE']],
  '689364338': [['chosong', 'JAPAN-NEXUS-DL']],
  '689362805': [['ahmullah', 'APT-DCSW']],
  '689369182': [['ktakahag', 'JAPAN-NEXUS']],
  '689370638': [['gapanton', 'APT-DCSW']],
  '689294727': [['jakeli', 'CX-APJC-SYD-DC-HTEC']],
  '689368439': [['gapanton', 'APT-DCSW']],
  '689361707': [['blockett', 'APT-DCSW']],
  '689302907': [['mahirani', 'HTTS-INDIA-DCSW']],
  '689358870': [['josliu', 'UC_WORKGROUP']],
  '689370671': [['ntshuma', 'APT-DCSW']],
  '689370714': [['jhouang', 'APT-DCSW']],
  '689335221': [['yondong', 'APT-GC-Nexus2']],
  '689370741': [['yondong', 'APT-GC-Nexus2']],
  '689370755': [['kkanchir', 'APT-DCSW']],
  '689370676': [['stayyar', 'APT-DCSW'], ['hagnanab', 'UC_WORKGROUP']],
  '689363887': [['gomamidi', 'HTTS-INDIA-DCSW']],
  '689367297': [['affkhan', 'APT-DCSW'], ['nidsaxen', 'UC_WORKGROUP']],
  '689362268': [['chaslu', 'APT-DCSW']],
  '689370502': [['musohail', 'APT-DCSW']],
  '68935

In [389]:
###### For FTS_Workgroup and UC_Workgroup Find previous accepted case to identify the workgroup, by the reverse date #######
def FindEngrWorkgroup(date,ccoid,AcceptEventsByDate):
    end_datetime = parser.parse('2020-02-27')
    cur_datetime = parser.parse(date)
    while cur_datetime > end_datetime:
        cur_date = cur_datetime.strftime("%Y-%m-%d")
        if cur_date in AcceptEventsByDate.keys():
            for caseno, caestakerlist in AcceptEventsByDate[cur_date].items():
                for casetaker in caestakerlist:
                    if ccoid == casetaker[0] and casetaker[1] != 'UC_WORKGROUP' and casetaker[1] != 'FTS_WORKGROUP':
                        #print("Found {} in workgroup {} on date {} from case {}".format(ccoid,casetaker[1],cur_date,caseno))
                        return ccoid, casetaker[1], cur_date
        cur_datetime = cur_datetime + datetime.timedelta(days=-1)
    return ccoid, "Unknown",date

FindEngrWorkgroup('2020-06-26','debabbar',RawAcceptEventsByDate)

('debabbar', 'Unknown', '2020-06-26')

In [217]:
###### Supressed : Fixed Workgroup will not be considered as the workgroup never can be known - for cross-skilling #####
###### 1. Only CaseInQueue and engineer takes it will take into account 2. Only APT-ACI-SOLUTIONS/2 yanked case(not InQueue) will take into account ######
def RemoveNotInQueueAcceptCase(start_date='2020-02-27',end_date=datetime.datetime.today().strftime("%Y-%m-%d"),RawAcceptEventsByDate={},InQueueEventsByDate={},InterestWorkgroup=['APT-ACI-SOLUTIONS','APT-ACI-SOLUTIONS2']):
    
    RemoveNotInQueueAcceptEventsByDate = copy.deepcopy(RawAcceptEventsByDate)
    for date,events in RawAcceptEventsByDate.items():
        if parser.parse(date) < parser.parse(start_date):
            continue
        elif parser.parse(date) > parser.parse(end_date):
            break
        InQueueCaseList = []
        RemoveCaseList = []
        for casetype,caselist in InQueueEventsByDate[date].items():
                InQueueCaseList = [*InQueueCaseList,*caselist]
        ###### Remove the case which is not in queue but taken and keep workgroup engineer yanked case by InterestWorkgroup######
        for caseno,casetakerlist in RawAcceptEventsByDate[date].items():
            #if date == '2020-06-20':
            #    print("{} {} {} {}".format(date,caseno,casetakerlist,InQueueCaseList))
            if caseno not in InQueueCaseList:
                ###### If there is a yank case which is not in queue, but yanked by APT-ACI-SOLUTIONS/2 engineer, it wont be removed ######.
                isYankedCase = False 
                for casetaker in casetakerlist:
                    #print("Case {} CaseTaker {}".format(caseno,casetaker))
                    if casetaker[1] in InterestWorkgroup:
                        #print("{} {} is a yanked case, keep in the accepted list".format(date,caseno))
                        isYankedCase = True
                if not isYankedCase:
                    #print("{} Removing the case {} as it is not the InQueue case".format(date,caseno))
                    RemoveCaseList.append(caseno)
        #if date == '2020-05-24':
            #print("Remove Case List {}".format(RemoveCaseList))
        for caseno in RemoveCaseList:
            del RemoveNotInQueueAcceptEventsByDate[date][caseno]
    
    UpdWGAcceptEventsByDate = copy.deepcopy(RemoveNotInQueueAcceptEventsByDate)
    #print("After Remove Case List {}".format(UpdWGAcceptEventsByDate['2020-05-24']))
    ###### Check UC_WORKGROUP and FTS_WORKGROUP Put in the right workgroup #######
    for date,caselist in RemoveNotInQueueAcceptEventsByDate.items():
        if parser.parse(date) < parser.parse(start_date):
            continue
        elif parser.parse(date) > parser.parse(end_date):
            break
        for caseno,casetakerlist in caselist.items():
            for idx,casetaker in enumerate(casetakerlist[:-1]):
                if casetaker[1] not in InterestWorkgroup:
                    ccoid,workgroup,date_wg = FindEngrWorkgroup(date,casetaker[0],RawAcceptEventsByDate)
                    if workgroup != "Unknown":
                        #print("caseno {} {} {} {}".format(caseno,ccoid,workgroup,date_wg))
                        UpdWGAcceptEventsByDate[date][caseno][idx][1] = workgroup
    #print("Return Accept case list by date {}".format(UpdWGAcceptEventsByDate['2020-05-24']))
    return UpdWGAcceptEventsByDate

date = '2020-05-24'
UpdWGAcceptEventsByDate = RemoveNotInQueueAcceptCase(start_date='2020-02-27',RawAcceptEventsByDate=RawAcceptEventsByDate,InQueueEventsByDate=InQueueEventsByDate)
UpdWGAcceptEventsByDate['2020-06-24'], len(UpdWGAcceptEventsByDate['2020-06-24'])

({'689352110': [['kikirov', 'SOF-SV'], ['fbegumr', 'GCE-APAC-SV']],
  '689352150': [['gagambhi', 'GCE-APAC-SV']],
  '689352135': [['donnylee', 'APT-SV']],
  '689333106': [['sairusse', 'APT-SV']],
  '689347849': [['lyxiao', 'APT-SV']],
  '689352202': [['davdodd', 'APT-SV']],
  '689352417': [['rkumarya', 'GCE-APAC-SV']],
  '689352639': [['dachandr', 'GCE-SV']],
  '689352699': [['vnandan', 'GCE-APAC-SV'], ['anahar', 'APT-SV']],
  '689351184': [['shajipas', 'GCE-APAC-SV']],
  '689212357': [['davdodd', 'APT-SV']],
  '689264575': [['frjin', 'APT-GC-UCS']],
  '689353157': [['aharalal', 'GCE-APAC-SV']],
  '689353213': [['rdevar', 'GCE-APAC-SV']],
  '689331025': [['nagordon', 'APT-SV']]},
 15)

In [390]:
def UpdWithEngrConvertInQueueEventsByDate(start_date='2020-02-27',end_date=datetime.datetime.today().strftime("%Y-%m-%d"),shift='apjc',RawAcceptEventsByDate={},InQueueEventsByDate={}):
    '''
    1.InQueue Case needs to find the accept engineer and its workgroup
    -- create new key 'AllWithEngr' to write to the same Dict InQueueEventsByDate
    '''
    
    start_datetime = parser.parse(start_date)
    end_datetime = parser.parse(end_date)
    cur_date = start_date
    cur_datetime = parser.parse(cur_date)
    
    while cur_datetime.date() <= end_datetime.date():
        cur_date = cur_datetime.strftime("%Y-%m-%d")
        if cur_date not in InQueueEventsByDate.keys():
            cur_datetime = cur_datetime + datetime.timedelta(days=1)
            continue

        ###### Function 1.InQueue Case needs to find the accept engineer and its workgroup ######
        if cur_date not in RawAcceptEventsByDate.keys():
            RawAcceptEventsByDate[cur_date] = {}
            
        InQueueEventsByDate[cur_date]['AllWithEngr'] = []
        for event in InQueueEventsByDate[cur_date]['All']:
            engr_line = []
            #print("RawEvent-{}".format(event)) #event is in order
            caseno, severity, casetype,casetime = event.split('-~')
            caseno_url = '<a {} href="http://mwz.cisco.com/{}" target="_blank">{}</a>'.format('style="text-decoration:none; color:#0000FF;"',caseno,caseno)
            _, UpdateTime = ACILib.isOnShift(timestamp=casetime,timezone = 'GMT',shift=shift)
            event_url = "{}-~{}-~{}-~{}".format(caseno_url,severity,casetype,UpdateTime)
            if caseno in RawAcceptEventsByDate[cur_date].keys():
                #print("CaseNo {} AccBy {}".format(caseno,RawAcceptEventsByDate[date][caseno]))
                #CaseNo 689154452 AccBy [['amikum', 'GCE-ACI-Solutions']]
                one_engr_line = ''
                for engr in RawAcceptEventsByDate[cur_date][caseno]:
                    if engr[1] == "Unknown" or 'FTS_WORKGROUP' in engr[1] or 'UC_WORKGROUP' in engr[1]:
                        #print("Date {} CCO {} WG {}".format(date,engr[0],engr[1]))
                        ccoid,new_wg,date = FindEngrWorkgroup(cur_date,engr[0],RawAcceptEventsByDate)
                        if new_wg != "Unknown":
                            engr[1] = new_wg
                    if not one_engr_line:
                        one_engr_line = one_engr_line+"{}({})".format(engr[0],engr[1])
                    else:
                        one_engr_line = one_engr_line+"-~"+"{}({})".format(engr[0],engr[1])
                engr_line = one_engr_line
                #print(engr_line)
            InQueueEventsByDate[cur_date]['AllWithEngr'].append("{}-~{}".format(event_url,engr_line))
            #print("Appending - UpdEventWithEngr-{}".format(events['AllWithEngr'][-1]))

        cur_datetime = cur_datetime + datetime.timedelta(days=1)
    
    return InQueueEventsByDate


InQueueEventsByDate = UpdWithEngrConvertInQueueEventsByDate(start_date='2020-06-26',shift=shift,RawAcceptEventsByDate=RawAcceptEventsByDate,InQueueEventsByDate=InQueueEventsByDate)
InQueueEventsByDate['2020-06-26']

{'All': ['689363887-~2-~FTS-~2020-06-26 00:00:12',
  '689361345-~1-~FTS-~2020-06-26 00:00:33',
  '689368439-~3-~WW-DCRS-~2020-06-26 00:00:48',
  '689351990-~2-~FTS-~2020-06-26 00:01:32',
  '689370502-~2-~FTS-~2020-06-26 00:04:56',
  '689361464-~1-~FTS-~2020-06-26 00:07:39',
  '689370638-~4-~WW-DCRS-~2020-06-26 00:17:22',
  '689361707-~3-~WW-DCRS-~2020-06-26 00:18:50',
  '689367297-~1-~WW-DCRS-~2020-06-26 00:25:02',
  '689313896-~3-~WW-DCRS-~2020-06-26 00:28:43',
  '689370256-~3-~WW-DCRS-~2020-06-26 00:30:07',
  '689370671-~3-~WW-DCRS-~2020-06-26 00:30:46',
  '689370676-~2-~WW-DCRS-~2020-06-26 00:31:29',
  '689313896-~3-~WW-DCRS-~2020-06-26 00:31:59',
  '689370676-~2-~WW-DCRS-~2020-06-26 00:37:41',
  '689370714-~2-~WW-DCRS-~2020-06-26 00:42:44',
  '689361657-~1-~UC-~2020-06-26 00:42:55',
  '689249756-~2-~WW-DCRS-~2020-06-26 00:46:06',
  '689370676-~2-~WW-DCRS-~2020-06-26 00:46:28',
  '689331072-~3-~WW-DCRS-~2020-06-26 00:47:31',
  '689370732-~3-~WW-DCRS-~2020-06-26 00:49:50',
  '6893707

In [391]:
###### Taken Case By Group ######
def CaseTakenByEngrGrpByDate(start_date='2020-06-17',end_date=datetime.datetime.today().strftime("%Y-%m-%d"),RawAcceptEventsByDate={}):
    
    CaseTakenByEngrByDate = {} # {'2020-50-23':{'APT-ACI-SOLUTIONS':{'siddhp':[689146430,689148487]}}}
    for date, items in RawAcceptEventsByDate.items():
        
        if parser.parse(date) < parser.parse(start_date):
            continue
        elif parser.parse(date) > parser.parse(end_date):
            break
            
        #print("Processing date {}".format(date))
        if date not in CaseTakenByEngrByDate.keys():
            CaseTakenByEngrByDate[date] = {}
        for caseno, casetakerlist in items.items():
            #print("Case {} - List {}".format(caseno,casetakerlist))
            for casetaker in casetakerlist:
                #print("Case {} - List {} Taker {}".format(caseno,casetakerlist,casetaker))
                if casetaker[1] not in CaseTakenByEngrByDate[date].keys():
                    CaseTakenByEngrByDate[date][casetaker[1]] = {}
                if casetaker[0] not in CaseTakenByEngrByDate[date][casetaker[1]].keys():
                    CaseTakenByEngrByDate[date][casetaker[1]][casetaker[0]] = []
                #print("Adding {} in {} {}".format(caseno,casetaker[1],casetaker[0]))
                CaseTakenByEngrByDate[date][casetaker[1]][casetaker[0]].append(caseno)
                
        #print(CaseTakenByEngrByDate['2020-06-24']['APT-SV'])
        Tmp_CaseTakenByEngrByDate = copy.deepcopy(CaseTakenByEngrByDate)
        
        ##### Move FTS_WORKGROUP an UC_WORKGRUOP case to InterestWorkgroup Case ######
        for casetype, takenlist in CaseTakenByEngrByDate[date].items():
            if casetype != 'UC_WORKGROUP' and casetype != 'FTS_WORKGROUP':
                continue
            #print("UC/FTS {} {} {}".format(date,casetype,takenlist))
            for ccoid, caselist in CaseTakenByEngrByDate[date][casetype].items():
                ccoid, new_wg, find_date = FindEngrWorkgroup(date,ccoid,RawAcceptEventsByDate)
                new_wg = new_wg.strip()
                if new_wg != "Unknown":
                    #print("ccoid {} in {} find new_wg: {} on {}".format(ccoid,casetype,new_wg,find_date))
                    if new_wg not in Tmp_CaseTakenByEngrByDate[date].keys():
                        #print("Adding new WG {}".format(new_wg))
                        Tmp_CaseTakenByEngrByDate[date][new_wg] = {}
                    if ccoid not in Tmp_CaseTakenByEngrByDate[date][new_wg].keys():
                        #print("Adding new WG {} new ccoid {}".format(new_wg,ccoid))
                        #print("Current -- {}".format(Tmp_CaseTakenByEngrByDate))
                        Tmp_CaseTakenByEngrByDate[date][new_wg][ccoid] = []
                    Tmp_CaseTakenByEngrByDate[date][new_wg][ccoid] = Tmp_CaseTakenByEngrByDate[date][new_wg][ccoid] + caselist
            #print(Tmp_CaseTakenByEngrByDate['2020-06-24']['APT-SV'])
            CaseTakenByEngrByDate = copy.deepcopy(Tmp_CaseTakenByEngrByDate)
            
            #ccoid,new_wg,date = FindEngrWorkgroup(date,engr[0],RawAcceptEventsByDate)
    return CaseTakenByEngrByDate

CaseTakenByEngrByDate = CaseTakenByEngrGrpByDate(start_date='2020-02-27',RawAcceptEventsByDate=RawAcceptEventsByDate)
CaseTakenByEngrByDate['2020-06-26']

{'CO-HARDWARE': {'sthomasg': ['689370542']},
 'APAC-GGN-HARDWARE': {'annagar': ['689370542']},
 'JAPAN-NEXUS-DL': {'chosong': ['689364338']},
 'APT-DCSW': {'ahmullah': ['689362805', '689370979'],
  'gapanton': ['689370638', '689368439', '689371074', '689371315'],
  'blockett': ['689361707', '689371801'],
  'ntshuma': ['689370671'],
  'jhouang': ['689370714'],
  'kkanchir': ['689370755', '689371077', '689371521'],
  'stayyar': ['689370676'],
  'hagnanab': ['689370676', '689352241', '689360144', '689352620'],
  'affkhan': ['689367297', '689322019', '689352930', '689371688'],
  'chaslu': ['689362268', '689304223', '689371246', '689371432', '689366236'],
  'musohail': ['689370502', '689371653'],
  'jleschni': ['689313896',
   '689370256',
   '689370732',
   '689371202',
   '689359745',
   '689349178',
   '689371383',
   '689241191',
   '689371831'],
  'josliu': ['689331072', '689359688', '689358870'],
  'joeunder': ['689370745', '689371018', '689371513'],
  'navchaud': ['689370210', '68937

In [392]:
###### Statstics Based on 1. CaseTakenByEngrByDate 2. InQueueEventsByDate
CaseTakenByEngrByDate['2020-06-26'],InQueueEventsByDate['2020-06-26']

({'CO-HARDWARE': {'sthomasg': ['689370542']},
  'APAC-GGN-HARDWARE': {'annagar': ['689370542']},
  'JAPAN-NEXUS-DL': {'chosong': ['689364338']},
  'APT-DCSW': {'ahmullah': ['689362805', '689370979'],
   'gapanton': ['689370638', '689368439', '689371074', '689371315'],
   'blockett': ['689361707', '689371801'],
   'ntshuma': ['689370671'],
   'jhouang': ['689370714'],
   'kkanchir': ['689370755', '689371077', '689371521'],
   'stayyar': ['689370676'],
   'hagnanab': ['689370676', '689352241', '689360144', '689352620'],
   'affkhan': ['689367297', '689322019', '689352930', '689371688'],
   'chaslu': ['689362268', '689304223', '689371246', '689371432', '689366236'],
   'musohail': ['689370502', '689371653'],
   'jleschni': ['689313896',
    '689370256',
    '689370732',
    '689371202',
    '689359745',
    '689349178',
    '689371383',
    '689241191',
    '689371831'],
   'josliu': ['689331072', '689359688', '689358870'],
   'joeunder': ['689370745', '689371018', '689371513'],
   'navch

In [393]:
def isFindWorkgroup(shift='apjc',tech_strip='aci',site='syd',workgroup='WW-ACI-SOLUTIONS',config={}):
    
    isFoundWorkgroup = False
    
    if shift == 'apjc':
        if site == 'syd':

            for wg in config[shift]['workgroup_syd_all'].split(','):
                if workgroup.lower() == wg.lower():
                    #print("Found syd workgroup from Syd workgroup {} - wg {}".format(workgroup,wg))
                    isFoundWorkgroup = True
                            
            #if tech_strip == 'aci' and not isFoundWorkgroup:
            #    #print("SYD WG {} EngrTakenCaseList {} ".format(workgroup,engrcasettakenlist))
            #    for wg in config[shift]['{}_{}_syd'.format('workgroup',tech_strip)].split(','):
            #        if workgroup.lower() == wg.lower():
            #            isFoundWorkgroup = True
            #            #print("Found syd workgroup from own workgroup {} - wg {}".format(workgroup,wg))
            #else:
            #    for wg in config[shift]['{}_{}'.format('workgroup',tech_strip)].split(','):
            #        if workgroup.lower() == wg.lower():
            #            isFoundWorkgroup = True

            if not isFoundWorkgroup and re.search(r'SYD',workgroup):
                    #print("Found syd keyword workgroup from Syd workgroup {} - wg {}".format(workgroup,wg))
                    isFoundWorkgroup = True
                    
        if site == 'blr':
            if tech_strip == 'aci':
                blr_wgs = config[shift]['{}_{}_blr'.format('workgroup',tech_strip)].split(',')
            else:
                blr_wgs = config[shift]['{}_{}'.format('workgroup',tech_strip)].split(',')
            for wg in blr_wgs:
                if workgroup.lower() == wg.lower():
                    isFoundWorkgroup = True
                    #print("Found blr workgroup from own workgroup {} - wg {}".format(workgroup,wg))
                        
    return isFoundWorkgroup

In [397]:
def GenerateCaseStatsDic(start_date='2020-06-09',end_date='2020-06-09',InQueueEventsByDate={},CaseTakenByEngrByDate={},AcceptCaseSevByDate={},InterestWorkgroup=[],shift='apjc',tech_strip='aci',config={}):
    
    '''
    InterestWorkgroup is used to generate Workgroup Stats
    tech_strip is used to generate specific Stats needed for that group
    '''
    CaseStatsByDateDic = {}
    
    start_datetime = parser.parse(start_date)
    end_datetime = parser.parse(end_date)
    cur_date = start_date
    cur_datetime = parser.parse(cur_date)
    
    while cur_datetime.date() <= end_datetime.date():
        cur_date = cur_datetime.strftime("%Y-%m-%d")
        if cur_date not in CaseStatsByDateDic.keys():
            #print("Generating CastStats Dic for date {}".format(cur_date))
            CaseStatsByDateDic[cur_date] = {}
        ###### Queue Case Volume Stats ######
        if cur_date not in InQueueEventsByDate.keys():
            cur_datetime = cur_datetime + datetime.timedelta(days=1)
            continue
        for workgroup,caselist in InQueueEventsByDate[cur_date].items():
            if workgroup == 'All' or workgroup == 'AllWithEngr' or workgroup == 'AllCases':
                continue
            #print("Date {} WG {} CaseList {}".format(cur_date,workgroup,caselist))
            if 'QueueCaseVol' not in CaseStatsByDateDic[cur_date].keys():
                CaseStatsByDateDic[cur_date]['QueueCaseVol'] = {}
            CaseStatsByDateDic[cur_date]['QueueCaseVol'][workgroup] = len(caselist)
        
        ###### Workgroup Stats ######
        ###### It differs from workgroup thus adding tech_strip to customzied it ######
        if 'WorkgroupCaseVol' not in CaseStatsByDateDic[cur_date].keys():
                CaseStatsByDateDic[cur_date]['WorkgroupCaseVol'] = {}
        if 'WorkgroupCase' not in CaseStatsByDateDic[cur_date].keys():
                CaseStatsByDateDic[cur_date]['WorkgroupCase'] = {}
        if 'WorkgroupEngrNo' not in CaseStatsByDateDic[cur_date].keys():
                CaseStatsByDateDic[cur_date]['WorkgroupEngrNo'] = {}
        if 'WorkgroupCaseVolBreakDown' not in CaseStatsByDateDic[cur_date].keys():
                CaseStatsByDateDic[cur_date]['WorkgroupCaseVolBreakDown'] = {}
        
        ##### Find Primary Workgroups of Sydney Workgroup of that tech_strip######
        if shift == 'apjc':
            CaseStatsByDateDic[cur_date]['SydEngrList'] = {}
            CaseStatsByDateDic[cur_date]['SydEngrList']['SydOnShiftEngrList'] = []
            CaseStatsByDateDic[cur_date]['SydEngrList']['SydOtherEngrList'] = []
            CaseStatsByDateDic[cur_date]['SydEngrList']['CrossSkill'] = {} ####Same as CaseStatsByDateDic {'WG_Name':{'ccoid1':[caselist],'ccoid2':[caselist]}
            
            TechStrip_Pri_WGs = []
            TechStrip_Blr_WGs = []
            if config[shift]['queuemode_{}'.format(tech_strip)] == 'dedicate':
                TechStrip_Pri_WGs = config[shift]['workgroup_{}'.format(tech_strip)]
                TechStrip_Blr_WGs = []
            elif config[shift]['queuemode_{}'.format(tech_strip)] == 'loadbalance':
                TechStrip_Pri_WGs = config[shift]['workgroup_{}_syd'.format(tech_strip)]
                TechStrip_Blr_WGs = [wg for wg in config[shift]['workgroup_{}_blr'.format(tech_strip)].split(',')]
                
            #print('Found Primary WG {} for tech_strip {}'.format(TechStrip_Pri_WGs,tech_strip))    
            TechStrip_Pri_WGs = [wg for wg in TechStrip_Pri_WGs.split(',')]
            syd_all_wgs = [wg for wg in config[shift]['workgroup_syd_all'].split(',')]
        
        ###### Calculate InterestWorkgroup Case Count ######
        for workgroup, takenlist in CaseTakenByEngrByDate[cur_date].items():
            
            #print("Shift:{} TechStrip:{} Workgroup:{} InterestWorkgroup:{}".format(shift,tech_strip,workgroup,InterestWorkgroup))
            if shift == 'apjc' and not (workgroup.upper() in [wg.upper() for wg in InterestWorkgroup] 
                                        or isFindWorkgroup(shift=shift,tech_strip=tech_strip,workgroup=workgroup,site='syd',config=config) 
                                        or isFindWorkgroup(shift=shift,tech_strip=tech_strip,workgroup=workgroup,site='blr',config=config)):
                continue
            elif shift == 'emea' and not workgroup.upper() in [wg.upper() for wg in InterestWorkgroup]:
                continue

            ###### Engineer Number Stats ######
            if shift == 'apjc':
                if workgroup in TechStrip_Pri_WGs:
                    #print("Primary workgroup {} takencaselist {} for tech_strip {}".format(workgroup,takenlist.keys(),tech_strip))
                    CaseStatsByDateDic[cur_date]['SydEngrList']['SydOnShiftEngrList'] = CaseStatsByDateDic[cur_date]['SydEngrList']['SydOnShiftEngrList'] + list(takenlist.keys())
                    #CaseStatsByDateDic[cur_date]['SydEngrList']['SydOnShiftEngrNo'] = len(takenlist)
                elif any(re.search(re.compile(wg),workgroup) for wg in syd_all_wgs) and not any(re.search(re.compile(wg),workgroup) for wg in TechStrip_Blr_WGs):
                    ###### Cross-Skill logic that need to find out all Syd_Other workgroup taken the case from current tech_strip ######
                    print("Found Other_WG {} TakenList {} in SYD_ALL of strip {}".format(workgroup,takenlist,tech_strip))
                    CaseStatsByDateDic[cur_date]['SydEngrList']['CrossSkill'][workgroup] = takenlist
                    #print("Generating CorssSkill List WG:{} TakenList:{}".format(workgroup,CaseStatsByDateDic[cur_date]['SydEngrList']['CrossSkill'][workgroup]))
                    CaseStatsByDateDic[cur_date]['SydEngrList']['SydOtherEngrList'] = CaseStatsByDateDic[cur_date]['SydEngrList']['SydOtherEngrList'] + list(takenlist.keys())
                
            #print("Hitting WG {} - {}".format(workgroup,takenlist))
            CaseStatsByDateDic[cur_date]['WorkgroupCase'][workgroup] = {}
            CaseStatsByDateDic[cur_date]['WorkgroupCase'][workgroup] = takenlist
            CaseStatsByDateDic[cur_date]['WorkgroupEngrNo'][workgroup] = len(takenlist)
            
            ###### Calculate workgroup case volume ######
            #print("WG {} Values {}".format(workgroup,CaseTakenByEngrByDate[cur_date][workgroup]))
            workgroup_caselist = CaseTakenByEngrByDate[cur_date][workgroup].values()
            workgroup_caselist = [case for cases in workgroup_caselist for case in cases]
            #print("WG {} CaseVolume {}".format(workgroup,workgroup_caselist))
            CaseStatsByDateDic[cur_date]['WorkgroupCaseVol'][workgroup] = len(workgroup_caselist)
            
            ###### Calculate workgroup case volume BreakDown P0,P1,P2,P3,P4######
            for case in workgroup_caselist:
                for sev,caselist in AcceptCaseSevByDate[cur_date].items():
                    if case in caselist:
                        #print("Date {} Case {} Sev {}".format(cur_date,case,sev))
                        if workgroup not in CaseStatsByDateDic[cur_date]['WorkgroupCaseVolBreakDown']:
                            CaseStatsByDateDic[cur_date]['WorkgroupCaseVolBreakDown'][workgroup] = [0,0,0,0,0]
                        CaseStatsByDateDic[cur_date]['WorkgroupCaseVolBreakDown'][workgroup][int(sev)] = CaseStatsByDateDic[cur_date]['WorkgroupCaseVolBreakDown'][workgroup][int(sev)]+1
        
        if shift == 'apjc':
            ###### Sorted by alphabet ######
            CaseStatsByDateDic[cur_date]['SydEngrList']['SydOnShiftEngrList'] = sorted(CaseStatsByDateDic[cur_date]['SydEngrList']['SydOnShiftEngrList'])
            CaseStatsByDateDic[cur_date]['SydEngrList']['SydOtherEngrList'] = sorted(CaseStatsByDateDic[cur_date]['SydEngrList']['SydOtherEngrList'])
            #print("Syd: Onshift {} Other".format(CaseStatsByDateDic[cur_date]['SydEngrList']['SydOnShiftEngrList'],CaseStatsByDateDic[cur_date]['SydEngrList']['SydOtherEngrList']))
            
            ###### Remove the Case which is not on InQueueEventsByDate[cur_date]['All'] list case and remaining are the Cross-Skill case ######
            removelist = {}
            print("XSkill List {}".format(CaseStatsByDateDic[cur_date]['SydEngrList']['CrossSkill']))
            for wg,takenlist in CaseStatsByDateDic[cur_date]['SydEngrList']['CrossSkill'].items():
                removelist[wg] = {}
                for ccoid,takenlist_1 in takenlist.items():
                    removelist[wg][ccoid] = []
                    removelist[wg][ccoid] = [caseno for caseno in takenlist_1 if caseno not in InQueueEventsByDate[cur_date]['AllCases']]
            print("NotInQueueCaseList {}".format(removelist))
            for wg, ccoid_case_list in removelist.items():
                for ccoid, caselist in ccoid_case_list.items():
                    for case in caselist:
                        CaseStatsByDateDic[cur_date]['SydEngrList']['CrossSkill'][wg][ccoid].remove(case)
                    if len(CaseStatsByDateDic[cur_date]['SydEngrList']['CrossSkill'][wg][ccoid]) == 0:
                        del CaseStatsByDateDic[cur_date]['SydEngrList']['CrossSkill'][wg][ccoid]
                if len(CaseStatsByDateDic[cur_date]['SydEngrList']['CrossSkill'][wg]) == 0:
                       del CaseStatsByDateDic[cur_date]['SydEngrList']['CrossSkill'][wg]
            print("XSkill List after removal {}".format(CaseStatsByDateDic[cur_date]['SydEngrList']['CrossSkill']))
            
        ###### It must before the LB Volume Calculation ######
        CaseStatsByDateDic[cur_date]['WorkgroupCaseVol']['Total'] = sum(CaseStatsByDateDic[cur_date]['WorkgroupCaseVol'].values())
        
        ###### Loadbalance Queue Case Volume ######
        if shift == 'apjc':
            if config[shift]['{}_{}'.format('queuemode',tech_strip)] == 'loadbalance':
                SYDCaseVol,BLRCaseVol = 0,0
                for workgroup, casevol in CaseStatsByDateDic[cur_date]['WorkgroupCaseVol'].items():
                    #print("WG {} CaseVolume {}".format(workgroup,casevol))
                    if isFindWorkgroup(shift=shift,tech_strip=tech_strip,workgroup=workgroup,site='syd',config=config):
                        SYDCaseVol = SYDCaseVol + casevol
                    elif isFindWorkgroup(shift=shift,tech_strip=tech_strip,workgroup=workgroup,site='blr',config=config):
                        BLRCaseVol = BLRCaseVol + casevol
                CaseStatsByDateDic[cur_date]['WorkgroupCaseVol']['SYDCaseVol'] = SYDCaseVol
                CaseStatsByDateDic[cur_date]['WorkgroupCaseVol']['BLRCaseVol'] = BLRCaseVol
                
        #print(CaseStatsByDateDic[cur_date])
        cur_datetime = cur_datetime + datetime.timedelta(days=1)
        
    return CaseStatsByDateDic

print("Interest Workgroup {}".format(InterestWorkgroup))
CaseStatsByDateDic = GenerateCaseStatsDic(start_date=start_date,end_date=end_date,InQueueEventsByDate=InQueueEventsByDate,
                                          CaseTakenByEngrByDate=CaseTakenByEngrByDate,AcceptCaseSevByDate=AcceptCaseSevByDate,
                                          InterestWorkgroup=InterestWorkgroup,
                                          shift=shift,tech_strip=tech_strip,config=config)
CaseStatsByDateDic['2020-06-24']

Interest Workgroup ['APT-DCSW']
Found Other_WG CX-APJC-SYD-DC-HTEC TakenList {'andylau': ['689275572']} in SYD_ALL of strip dcrs
Found Other_WG APT-ACI-SOLUTIONS TakenList {'minkwong': ['689275606']} in SYD_ALL of strip dcrs
XSkill List {'CX-APJC-SYD-DC-HTEC': {'andylau': ['689275572']}, 'APT-ACI-SOLUTIONS': {'minkwong': ['689275606']}}
NotInQueueCaseList {'CX-APJC-SYD-DC-HTEC': {'andylau': ['689275572']}, 'APT-ACI-SOLUTIONS': {'minkwong': []}}
XSkill List after removal {'APT-ACI-SOLUTIONS': {'minkwong': ['689275606']}}
XSkill List {}
NotInQueueCaseList {}
XSkill List after removal {}
Found Other_WG APT-ACI-SOLUTIONS TakenList {'junwa': ['689186391', '689284505']} in SYD_ALL of strip dcrs
Found Other_WG CX-APJC-SYD-DC-HTEC TakenList {'andylau': ['689281255', '689278940', '689284515', '689259201']} in SYD_ALL of strip dcrs
XSkill List {'APT-ACI-SOLUTIONS': {'junwa': ['689186391', '689284505']}, 'CX-APJC-SYD-DC-HTEC': {'andylau': ['689281255', '689278940', '689284515', '689259201']}}
Not

{'QueueCaseVol': {'FTS': 4, 'WW-DCRS': 46, 'UC': 5},
 'WorkgroupCaseVol': {'APT-DCSW': 44, 'CX-APJC-SYD-DC-HTEC': 3, 'Total': 47},
 'WorkgroupCase': {'APT-DCSW': {'musohail': ['689350605',
    '689273314',
    '689352893'],
   'joeunder': ['689333366', '689352620'],
   'jaehwlee': ['689340085', '689348906', '689353337'],
   'blockett': ['689293916', '689352274', '689352979', '689353269'],
   'jhouang': ['689352249'],
   'zeemalik': ['689352241', '689352215'],
   'hagnanab': ['689348944', '689250317'],
   'stayyar': ['689350937'],
   'gapanton': ['689349481', '689352987'],
   'jleschni': ['689352221', '689352312', '689352527', '689352618'],
   'haohe': ['689352222', '689339493', '689343053'],
   'chaslu': ['689352200',
    '689352442',
    '689353261',
    '689353434',
    '689314937',
    '689320501'],
   'gurswami': ['689351990', '689350244', '689352010'],
   'ntshuma': ['689352420', '689180151', '689333366'],
   'mabushei': ['689127204'],
   'kmajewsk': ['689350238', '689352980'],
  

In [369]:
import pathlib

#reportpath = config[shift]['reportpath_aci']
#
    
def MakeHTMLTrackerReport(start_date='2020-06-09',end_date='2020-06-09',InQueueEventsByDate={},CaseTakenByEngrByDate={},CaseStatsByDateDic={},DispatchEvents=[],shift='apjc',tech_strip='aci',config={}):
    '''
    Generate Deaily HTML Report
    '''
    
    start_datetime = parser.parse(start_date)
    end_datetime = parser.parse(end_date)
    cur_date = start_date
    cur_datetime = parser.parse(cur_date)
    HTMLTrackingReport = {}

    QueueName = config[shift]['_'.join(['queuename',tech_strip.upper()])].split(',')
    QueueName = [qn.upper() for qn in QueueName]
        
    while cur_datetime.date() <= end_datetime.date():
        
        cur_date = cur_datetime.strftime("%Y-%m-%d")
        if cur_date not in InQueueEventsByDate.keys() or cur_date not in CaseTakenByEngrByDate or cur_date not in CaseStatsByDateDic:
            HTMLTrackingReport[cur_date] = '{} {} {} No Event. <br> Possible Reason: <br>1.Shift has not started or just started.<br>2. BDB is down.'.format(cur_date,shift.upper(),tech_strip.upper(),)
            cur_datetime = cur_datetime + datetime.timedelta(days=1)
            continue
        
        ###### Start Generating the Daily HTML Report ######
        
        #print(InQueueEventsByDate[cur_date])
        #print(CaseTakenByEngrByDate[cur_date])
        #print(CaseStatsByDateDic[cur_date])
        
        if shift == 'apjc':
            dailyreportstring = "{} Queue Case Volume -- Total {} *** Update at(AEST/AEDT): {}\n".format(cur_date, sum(CaseStatsByDateDic[cur_date]['QueueCaseVol'].values()),CaseStatsByDateDic['UpdateTime'])
        elif shift == 'emea':
            dailyreportstring = "{} Queue Case Volume -- Total {} *** Update at(IST): {}\n".format(cur_date, sum(CaseStatsByDateDic[cur_date]['QueueCaseVol'].values()),CaseStatsByDateDic['UpdateTime'])
            
        dailyreportstring = dailyreportstring+"==================\n"
        for queue, volume in CaseStatsByDateDic[cur_date]['QueueCaseVol'].items():
            dailyreportstring = dailyreportstring+"{}:{}\n".format(queue,volume)
        dailyreportstring = dailyreportstring+'\n'
        
        if shift == 'apjc':
            if tech_strip =='aci' or tech_strip == 'sec': ##### loadbalance report#####
                
                dailyreportstring = dailyreportstring+"{} Sydney SR ownership taken by workgroup - {} cases \n".format(cur_date,CaseStatsByDateDic[cur_date]['WorkgroupCaseVol']['SYDCaseVol'])
                dailyreportstring = dailyreportstring+"==================\n"
                
                ##### list 5 engineers per line
                #onshiftengineerlist = []
                #curengrlist = []
                #for engineer in CaseStatsByDateDic[cur_date]['SydEngrList']['SydOnShiftEngrList'].split(','):
                #    curengrlist.append(engineer)
                #    if len(curengrlist) == 5:
                #        onshiftengineerlist.append(', '.join(curengrlist))
                #        curengrlist = []
                #else:
                #    onshiftengineerlist.append(', '.join(curengrlist))
                #    
                dailyreportstring = dailyreportstring+"OnShift Engineer({}): {}\n".format(len(CaseStatsByDateDic[cur_date]['SydEngrList']['SydOnShiftEngrList']),
                                                                                          ', '.join(CaseStatsByDateDic[cur_date]['SydEngrList']['SydOnShiftEngrList']))
                #for engrlist in onshiftengineerlist[1:]:
                #    dailyreportstring = dailyreportstring+engrlist+"\n"
                #    
                dailyreportstring = dailyreportstring+"Other Engineer({}): {}\n\n".format(len(CaseStatsByDateDic[cur_date]['SydEngrList']['SydOtherEngrList']),
                                                                                          ', '.join(CaseStatsByDateDic[cur_date]['SydEngrList']['SydOtherEngrList']))

                for workgroup,engrcasettakenlist in CaseStatsByDateDic[cur_date]['WorkgroupCase'].items():
                    if not isFindWorkgroup(shift=shift,tech_strip=tech_strip,workgroup=workgroup,site='syd',config=config):
                        continue
                    dailyreportstring = dailyreportstring + "WORKGROUP {} - {} engineers {} cases\n".format(workgroup,CaseStatsByDateDic[cur_date]['WorkgroupEngrNo'][workgroup],CaseStatsByDateDic[cur_date]['WorkgroupCaseVol'][workgroup])
                    dailyreportstring = dailyreportstring + "Case Taken by Severity P1:{} P2:{} P3:{} P4:{}\n".format(*CaseStatsByDateDic[cur_date]['WorkgroupCaseVolBreakDown'][workgroup][1:])
                    for engr, caselist in engrcasettakenlist.items():
                        #print("Engr {} CaseSum {} CaseList {}".format(engr,len(caselist),caselist))
                        dailyreportstring = dailyreportstring+"{} {} {}\n".format(engr,len(caselist),caselist)
                    dailyreportstring = dailyreportstring+"\n"
                    
                dailyreportstring = dailyreportstring+"{} Bangalore SR ownership taken by workgroup - {} cases \n".format(cur_date,CaseStatsByDateDic[cur_date]['WorkgroupCaseVol']['BLRCaseVol'])
                dailyreportstring = dailyreportstring+"==================\n"
                for workgroup,engrcasettakenlist in CaseStatsByDateDic[cur_date]['WorkgroupCase'].items():
                    #print("WG {} EngrTakenCaseList {} ".format(workgroup,engrcasettakenlist))
                    if not isFindWorkgroup(shift=shift,tech_strip=tech_strip,workgroup=workgroup,site='blr',config=config):
                        continue
                    dailyreportstring = dailyreportstring + "WORKGROUP {} - {} engineers {} cases\n".format(workgroup,CaseStatsByDateDic[cur_date]['WorkgroupEngrNo'][workgroup],CaseStatsByDateDic[cur_date]['WorkgroupCaseVol'][workgroup])
                    dailyreportstring = dailyreportstring + "Case Taken by Severity P1:{} P2:{} P3:{} P4:{}\n".format(*CaseStatsByDateDic[cur_date]['WorkgroupCaseVolBreakDown'][workgroup][1:])
                    for engr, caselist in engrcasettakenlist.items():
                        #print("Engr {} CaseSum {} CaseList {}".format(engr,len(caselist),caselist))
                        dailyreportstring = dailyreportstring+"{} {} {}\n".format(engr,len(caselist),caselist)
                    dailyreportstring = dailyreportstring+"\n"
                    
            else:
                
                dailyreportstring = dailyreportstring+"{} SR ownership taken by workgroup - {} cases \n".format(cur_date,CaseStatsByDateDic[cur_date]['WorkgroupCaseVol']['Total'])
                dailyreportstring = dailyreportstring+"==================\n"
                for workgroup,engrcasettakenlist in CaseStatsByDateDic[cur_date]['WorkgroupCase'].items():
                    #print("WG {} EngrTakenCaseList {} ".format(workgroup,engrcasettakenlist))
                    dailyreportstring = dailyreportstring + "WORKGROUP {} - {} engineers {} cases\n".format(workgroup,CaseStatsByDateDic[cur_date]['WorkgroupEngrNo'][workgroup],CaseStatsByDateDic[cur_date]['WorkgroupCaseVol'][workgroup])
                    dailyreportstring = dailyreportstring + "Case Taken by Severity P1:{} P2:{} P3:{} P4:{}\n".format(*CaseStatsByDateDic[cur_date]['WorkgroupCaseVolBreakDown'][workgroup][1:])
                    for engr, caselist in engrcasettakenlist.items():
                        #print("Engr {} CaseSum {} CaseList {}".format(engr,len(caselist),caselist))
                        dailyreportstring = dailyreportstring+"{} {} {}\n".format(engr,len(caselist),caselist)
                    dailyreportstring = dailyreportstring+"\n"
                    
        elif shift == 'emea':
            
            dailyreportstring = dailyreportstring+"{} SR ownership taken by workgroup - {} cases.\n".format(cur_date,CaseStatsByDateDic[cur_date]['WorkgroupCaseVol']['Total'])
            dailyreportstring = dailyreportstring+"==================\n"
            for workgroup,engrcasettakenlist in CaseStatsByDateDic[cur_date]['WorkgroupCase'].items():
                #print("WG {} EngrTakenCaseList {} ".format(workgroup,engrcasettakenlist))
                dailyreportstring = dailyreportstring + "WORKGROUP {} - {} engineers {} cases\n".format(workgroup,CaseStatsByDateDic[cur_date]['WorkgroupEngrNo'][workgroup],CaseStatsByDateDic[cur_date]['WorkgroupCaseVol'][workgroup])
                dailyreportstring = dailyreportstring + "Case Taken by Severity P1:{} P2:{} P3:{} P4:{}\n".format(*CaseStatsByDateDic[cur_date]['WorkgroupCaseVolBreakDown'][workgroup][1:])
                for engr, caselist in engrcasettakenlist.items():
                    #print("Engr {} CaseSum {} CaseList {}".format(engr,len(caselist),caselist))
                    dailyreportstring = dailyreportstring+"{} {} {}\n".format(engr,len(caselist),caselist)
                dailyreportstring = dailyreportstring+"\n"
                
        #dailyreportstring = dailyreportstring+"\n"
        dailyreportstring = dailyreportstring+"{} Realtime events - Total Cases(incl. FTS/UC): {}\n Queue: {} \n".format(cur_date,sum(CaseStatsByDateDic[cur_date]['QueueCaseVol'].values()),", ".join(QueueName))
        dailyreportstring = dailyreportstring+"==================\n"
        dailyreportstring = dailyreportstring+"\n".join(InQueueEventsByDate[cur_date]['AllWithEngr'])
        dailyreportstring = dailyreportstring+"\n\n"
        
        ###### List Dispatched Case ######
        dailyreportstring = dailyreportstring+"Possible dispatched cases - next 14 days\n"
        dailyreportstring = dailyreportstring+"==================\n"
        furturedispatchevents = []
        for dispatchevent in DispatchEvents[::-1]:
            if not dispatchevent: ###### Possible emptyline at the bottomr ######
                continue 
            #print("Dispatch event {}".format(dispatchevent))
            caseno,casesev,casetime,casetitle = dispatchevent.split('-~')
            caseno_url = '<a {} href="http://mwz.cisco.com/{}" target="_blank">{}</a>'.format('style="text-decoration:none; color:#0000FF;"',caseno,caseno)
            casedatetime = parser.parse(casetime)
            if  casedatetime >= parser.parse(cur_date): 
                ###### Only list next 2 weeks dispatch case ######
                if parser.parse(casetime) <= parser.parse(cur_date)+datetime.timedelta(days=14):
                    furturedispatchevents.append("-~".join([caseno_url,casesev,casetime,casetitle]))
            else: ######Events in order and break when before the current date######
                break
        for event in furturedispatchevents[::-1]:
            dailyreportstring = dailyreportstring+event+"\n"
        
        #print(dailyreportstring)
        cur_datetime = cur_datetime + datetime.timedelta(days=1)
        dailyreportstringhtml = dailyreportstring.replace('\n','<br>\n')
        HTMLTrackingReport[cur_date] = dailyreportstringhtml
        
    return HTMLTrackingReport

CaseStatsByDateDic['UpdateTime'] = 'Fixed'
HTMLTrackingReport = MakeHTMLTrackerReport(start_date='2020-06-26',end_date='2020-06-26',InQueueEventsByDate=InQueueEventsByDate,CaseTakenByEngrByDate=CaseTakenByEngrByDate,
                                           CaseStatsByDateDic=CaseStatsByDateDic,shift=shift,tech_strip=tech_strip,config=config)
HTMLTrackingReport['2020-06-26'].split('\n')

['2020-06-26 Queue Case Volume -- Total 13 *** Update at(IST): Fixed<br>',
 '==================<br>',
 'FTS:5<br>',
 'EME-ACI-Solutions:7<br>',
 'UC:1<br>',
 '<br>',
 '2020-06-26 SR ownership taken by workgroup - 5 cases.<br>',
 '==================<br>',
 'WORKGROUP GCE-ACI-Solutions - 5 engineers 5 cases<br>',
 'Case Taken by Severity P1:0 P2:0 P3:4 P4:1<br>',
 "njaved 1 ['689371904']<br>",
 "birai 1 ['689364996']<br>",
 "visgupt2 1 ['689372358']<br>",
 "deseth2 1 ['689355508']<br>",
 "deepakba 1 ['689212660']<br>",
 '<br>',
 '2020-06-26 Realtime events - Total Cases(incl. FTS/UC): 13<br>',
 ' Queue: EME-ACI-SOLUTIONS <br>',
 '==================<br>',
 '<a style="text-decoration:none; color:#0000FF;" href="http://mwz.cisco.com/689226567" target="_blank">689226567</a>-~3-~FTS-~2020-06-26 22:01:04-~onaguib(EMEAR-ORCH-ACI)<br>',
 '<a style="text-decoration:none; color:#0000FF;" href="http://mwz.cisco.com/689371455" target="_blank">689371455</a>-~3-~FTS-~2020-06-26 22:02:37-~[]<br>',
 '<a